In [1]:
import pandas as pd 
import yfinance as yf
import numpy as np
from scipy.stats import kurtosis, skew
import openpyxl


In [2]:
def fetch_yahoo_data(ticker: str, period: str = "1y", interval: str = "1d") -> pd.DataFrame:
    """
    Busca dados históricos de OHLCV para o ticker especificado no Yahoo Finance.

    Parâmetros:
    - ticker: código do ativo (ex: "VALE3.SA" para ações brasileiras)
    - period: período de dados (ex: "180d", "1y", "2y")
    - interval: intervalo de tempo (ex: "1d", "1wk", "1mo")

    Retorna:
    - DataFrame com colunas: Open, High, Low, Close, Adj Close e Volume
    """
    df = yf.download(ticker,
                     period=period,
                     interval=interval,
                     auto_adjust=True)  # ajusta splits/dividends
    return df


In [23]:
weights = {

    "NU": 0.25,
    "SOFI": 0.25,
    "XYZ": 0.25,
    "HOOD": 0.25


}
tickers = weights.keys()
time = "30y"
risk_threshold = -0.01

In [4]:
def fetch_data(ticker):
    df = fetch_yahoo_data(ticker, period=time, interval="1d")
    df = df.xs(ticker, axis=1, level='Ticker')

    # PnL diário (retorno)
    df[f"pnl_{ticker}"] = df['Close'].pct_change()

    # Volume relativo
    df[f"volrel_{ticker}"] = df['Volume'] / df['Volume'].rolling(window=10).mean()

    # Amplitude média (High - Low)
    df[f"amp_{ticker}"] = (df["High"] - df["Low"]).rolling(window=10).mean()

    # Gaps > 1%
    gaps = (df["Open"].shift(-1) - df["Close"]) / df["Close"]
    df[f"gaps_{ticker}"] = gaps.rolling(window=10).apply(lambda x: (np.abs(x) > 0.01).sum(), raw=True)

    # Manter apenas colunas relevantes
    metric_cols = [col for col in df.columns if col.startswith((
        'pnl_', 
        'volrel_', 
        # 'amp_', 
        # 'gaps_'
        ))]
    df = df.dropna(subset=metric_cols)

    return df[metric_cols]


### Métricas

In [5]:
# pnl acumulado em janela de 5 dias
def calculate_metrics(df):
    # label de classificacao de risco
    df["pnl_5d"] = (
        (1 + df["PnL"])[::-1]
        .rolling(window=5)
        .apply(lambda x: np.prod(x) - 1, raw=True)
    )[::-1]
    df['risk'] = df.apply(lambda x: True if x['pnl_5d'] < risk_threshold else False, axis=1)
    df = df.drop(columns=["pnl_5d"]) 

    # pnl acumulado dos últimos 5 dias
    df["pnl_cum"] = df["PnL"].rolling(window=5).apply(np.prod)

    # volatility
    df['volatility'] = df['PnL'].rolling(window=20).std()

    # skewness
    df['skewness'] = df['PnL'].rolling(window=20).apply(lambda x: skew(x), raw=False)

    # kurtosis
    df["kurtosis"] = df["PnL"].rolling(window=20).apply(lambda x: kurtosis(x), raw=False)

    # sharpe ratio
    df["ret_medio"] = df["PnL"].rolling(window=20).mean()
    df['sharpe'] = df["ret_medio"] / df["volatility"]
    df = df.drop(columns=["ret_medio"])
    return df

In [24]:
portfolio = pd.DataFrame()
for ticker in tickers:
    data = fetch_data(ticker)
    portfolio = pd.concat([portfolio, data], axis=1)
    portfolio = portfolio.dropna()
portfolio
pnl_cols = [col for col in portfolio.columns if col.startswith('pnl_')]
portfolio['PnL'] = portfolio[pnl_cols].apply(lambda x: x.mul(weights[x.name.split('_')[1]]), axis=0).sum(axis=1)
portfolio = portfolio.drop(columns=pnl_cols)
portfolio = calculate_metrics(portfolio) 
portfolio = portfolio.dropna()


volrel_cols = [col for col in portfolio.columns if col.startswith('volrel_')]
for i in range (1,len(volrel_cols)+1):
    portfolio[f'volrel.{i}'] = portfolio[volrel_cols[i-1]]
for i in range(1,8):
    portfolio[f'PnL.{i}'] = portfolio['PnL']
for i in range(1,3):
    portfolio[f'sharpe.{i}'] = portfolio['sharpe']
portfolio = portfolio.reset_index()
portfolio = portfolio.drop(columns=['pnl_cum','PnL','sharpe'] + volrel_cols)
portfolio

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


Price,Date,risk,volatility,skewness,kurtosis,volrel.1,volrel.2,volrel.3,volrel.4,PnL.1,PnL.2,PnL.3,PnL.4,PnL.5,PnL.6,PnL.7,sharpe.1,sharpe.2
0,2022-01-20,True,0.033626,-0.073865,-0.769748,1.341718,3.685298,0.954544,0.944985,0.022662,0.022662,0.022662,0.022662,0.022662,0.022662,0.022662,-0.256114,-0.256114
1,2022-01-21,True,0.035750,-0.041596,-0.999658,0.873875,1.656398,1.043385,1.261472,-0.062935,-0.062935,-0.062935,-0.062935,-0.062935,-0.062935,-0.062935,-0.308724,-0.308724
2,2022-01-24,True,0.034529,0.083409,-0.804823,1.193829,1.476691,1.849231,1.818186,-0.018900,-0.018900,-0.018900,-0.018900,-0.018900,-0.018900,-0.018900,-0.388562,-0.388562
3,2022-01-25,False,0.034439,0.117936,-0.777690,0.842748,0.756018,1.022393,0.926047,-0.010237,-0.010237,-0.010237,-0.010237,-0.010237,-0.010237,-0.010237,-0.401163,-0.401163
4,2022-01-26,False,0.034397,0.115774,-0.770096,0.932004,0.818277,0.891186,1.128427,-0.036903,-0.036903,-0.036903,-0.036903,-0.036903,-0.036903,-0.036903,-0.399972,-0.399972
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,2025-06-06,False,0.022471,0.134670,-0.238315,0.809955,1.142637,0.924470,1.790788,0.024885,0.024885,0.024885,0.024885,0.024885,0.024885,0.024885,0.354589,0.354589
848,2025-06-09,False,0.022763,0.188536,-0.345347,0.990560,1.094424,0.868540,1.615468,-0.008862,-0.008862,-0.008862,-0.008862,-0.008862,-0.008862,-0.008862,0.323960,0.323960
849,2025-06-10,False,0.019938,0.001068,-0.342496,0.823916,0.984331,1.245869,0.940802,-0.002005,-0.002005,-0.002005,-0.002005,-0.002005,-0.002005,-0.002005,0.228168,0.228168
850,2025-06-11,False,0.018966,-0.216443,-0.554797,1.710010,1.311574,0.804019,0.787437,0.034682,0.034682,0.034682,0.034682,0.034682,0.034682,0.034682,0.213159,0.213159


In [27]:
portfolio[portfolio['risk'] == False]
trues = portfolio[portfolio['risk'] == True]
trues_size = len(trues)
falses = portfolio[portfolio['risk'] == False]
falses 

Price,Date,risk,volatility,skewness,kurtosis,volrel.1,volrel.2,volrel.3,volrel.4,PnL.1,PnL.2,PnL.3,PnL.4,PnL.5,PnL.6,PnL.7,sharpe.1,sharpe.2
3,2022-01-25,False,0.034439,0.117936,-0.777690,0.842748,0.756018,1.022393,0.926047,-0.010237,-0.010237,-0.010237,-0.010237,-0.010237,-0.010237,-0.010237,-0.401163,-0.401163
4,2022-01-26,False,0.034397,0.115774,-0.770096,0.932004,0.818277,0.891186,1.128427,-0.036903,-0.036903,-0.036903,-0.036903,-0.036903,-0.036903,-0.036903,-0.399972,-0.399972
5,2022-01-27,False,0.035845,0.178418,-0.949770,1.105934,0.776561,1.177581,1.737128,-0.058798,-0.058798,-0.058798,-0.058798,-0.058798,-0.058798,-0.058798,-0.443273,-0.443273
6,2022-01-28,False,0.033874,0.034366,-1.067651,0.607477,0.696859,0.888690,2.495350,0.024533,0.024533,0.024533,0.024533,0.024533,0.024533,0.024533,-0.505230,-0.505230
7,2022-01-31,False,0.044344,0.872174,0.856535,0.839903,0.624088,0.781405,1.672722,0.110758,0.110758,0.110758,0.110758,0.110758,0.110758,0.110758,-0.244638,-0.244638
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
847,2025-06-06,False,0.022471,0.134670,-0.238315,0.809955,1.142637,0.924470,1.790788,0.024885,0.024885,0.024885,0.024885,0.024885,0.024885,0.024885,0.354589,0.354589
848,2025-06-09,False,0.022763,0.188536,-0.345347,0.990560,1.094424,0.868540,1.615468,-0.008862,-0.008862,-0.008862,-0.008862,-0.008862,-0.008862,-0.008862,0.323960,0.323960
849,2025-06-10,False,0.019938,0.001068,-0.342496,0.823916,0.984331,1.245869,0.940802,-0.002005,-0.002005,-0.002005,-0.002005,-0.002005,-0.002005,-0.002005,0.228168,0.228168
850,2025-06-11,False,0.018966,-0.216443,-0.554797,1.710010,1.311574,0.804019,0.787437,0.034682,0.034682,0.034682,0.034682,0.034682,0.034682,0.034682,0.213159,0.213159


In [28]:
portfolio.to_parquet('portfolio1.parquet')

In [16]:
len(portfolio[portfolio['risk'] == False])/len(portfolio[portfolio['risk'] == True])

1.9948248407643312

In [108]:
portfolios = [pd.read_parquet(f'portfolio{i}.parquet') for i in range(0,6)]
test_portfolio = pd.concat(portfolios)
test_portfolio.to_parquet('test_portfolio.parquet')


In [110]:
portfolio = pd.read_parquet('portfolio5.parquet')
portfolio[portfolio['risk'] == True]

Price,Date,risk,volatility,skewness,kurtosis,volrel.1,volrel.2,volrel.3,volrel.4,PnL.1,PnL.2,PnL.3,PnL.4,PnL.5,PnL.6,PnL.7,sharpe.1,sharpe.2
2,2008-05-05,True,0.020489,1.160829,0.930849,0.302640,0.693627,0.969898,0.686597,-0.001037,-0.001037,-0.001037,-0.001037,-0.001037,-0.001037,-0.001037,0.185518,0.185518
3,2008-05-06,True,0.020671,1.200535,0.926770,1.119575,0.641999,0.902889,0.720533,-0.008452,-0.008452,-0.008452,-0.008452,-0.008452,-0.008452,-0.008452,0.157462,0.157462
5,2008-05-08,True,0.020447,1.175908,0.962302,0.817946,0.587582,0.883512,0.792481,0.000075,0.000075,0.000075,0.000075,0.000075,0.000075,0.000075,0.185061,0.185061
6,2008-05-09,True,0.021419,0.924028,0.780012,0.758427,0.858514,0.822590,0.810020,-0.032416,-0.032416,-0.032416,-0.032416,-0.032416,-0.032416,-0.032416,0.145831,0.145831
10,2008-05-15,True,0.020317,0.755638,1.004363,1.289554,0.425859,0.857059,0.426930,-0.002357,-0.002357,-0.002357,-0.002357,-0.002357,-0.002357,-0.002357,0.297144,0.297144
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4094,2025-04-03,True,0.012964,0.853189,0.764080,1.441481,1.836258,1.186398,1.128525,0.011699,0.011699,0.011699,0.011699,0.011699,0.011699,0.011699,0.225221,0.225221
4095,2025-04-04,True,0.016802,-0.405359,1.729760,1.739876,1.267371,1.492539,1.787952,-0.045518,-0.045518,-0.045518,-0.045518,-0.045518,-0.045518,-0.045518,0.013282,0.013282
4123,2025-05-19,True,0.007489,0.296406,-0.402609,0.566162,0.332595,1.419414,0.968266,0.003450,0.003450,0.003450,0.003450,0.003450,0.003450,0.003450,0.679276,0.679276
4124,2025-05-20,True,0.007447,0.418269,-0.287798,0.711779,0.389442,0.837707,0.972216,0.002867,0.002867,0.002867,0.002867,0.002867,0.002867,0.002867,0.642234,0.642234


In [11]:
portfolio.to_excel('portfolio.xlsx')

In [ ]:
df = pd.read_excel('portfolio6.xlsx')
df.to_parquet('portfolio6.parquet')
df

In [ ]:
trues = portfolio[portfolio['risk'] == True]
falses = portfolio[portfolio['risk'] == False]
print(len(trues), len(falses))
